# Práctica minería de texto

* [Presentación](#opcion-1)
* [Técnicas utilizadas](#opcion-2)
    * [Limpieza de texto](#opcion-2-1)
    * [Stemming](#opcion-2-2)
    * [Lematización](#opcion-2-3)
    * [Entidades nombradas(*ENR*)](#opcion-2-4)
* [Solución](#opcion-3)
    * [Extracción de la información](#opcion-3-1)
    * [Tratamiento de los textos](#opcion-3-2)
    * [Agrupación y resultado](#opcion-3-3)
* [Conclusiones](#opcion-4)


### Presentación <a class="anchor" id="opcion-1"></a>

El ejercicio consiste en el tratamiento de una serie de noticias, descargadas de distintos diarios electrónicos, y que deben se agrupadas de forma automática según su contenido.

Para ello hay que hacer uso de distintas técnicas de tratamiento de lenguaje natural: limpieza del texto, stemming, lematizado, tratamiento de entiidades nombradas, etc.

Se parte de un código fuente en el que teniendo disponibles los fichero .txt, se encarga de realizar las agrupación de los textos y compararlo con un array de valores en el que se representa el resultado ideal.
    

### Técnicas utilizadas <a class="anchor" id="opcion-2"></a>

Para la realización del ejercicio se han utilizados las siguientes técnicas de tratamiento de lenguaje natural, haciendo uso de la librería *Python* **NLTK**:

#### - Limpieza del texto: <a class="anchor" id="opcion-2-1"></a> 

En este paso se elimina del texto original los signos de puntuación y aquellas palabras que no aportarán información a los pasos posteriores, denominadas *stopwords*.

#### - Stemming: <a class="anchor" id="opcion-2-2"></a>

Es un método para reducir una palabra a su raíz, haciendo que un tratamiento de búsqueda o agrupación posterior considere dos palabras que tienen esa raíz común como la misma.

#### - Lematización: <a class="anchor" id="opcion-2-3"></a>

   Método por el cual se simplifica una *forma flexionada* (plural, femenino, conjugada, ...) y sea sustituida por la forma que por norma es aceptada como representación de todas ellas, es decir, la forma que podríamos encontrar en cualquier diccionario.

#### - Entidades nombradas (*ENR*): <a class="anchor" id="opcion-2-4"></a>

   Unidad de información fundamental que se refiere a nombres propios que pueden ser clasificados en categorías variadas.
    
   Las principales categorías son: Personas, Lugares y Organizaciones, si bien pueden aparecer mas segun el tipo de dato (fechas) o el dominio del texto (político, farmacéutico, ...)

### Solución <a class="anchor" id="opcion-3"></a>

#### - Extración de la información <a class="anchor" id="opcion-3-1"></a>
    
Como primera acción se ha realizado un proceso de extracción de la información desde los fichero *HTML* a *txt*.
Haciendo uso de la librería *BeautifulSoup* se han realizan los siguientes pasos:
- Identificamos el origen. dado que los fichero están descargado y no tenemosla URL, se hace uso de la metainformación guarda en el propio HTML haciendo referencia al origen del mismo:

In [ ]:
         
    origen = bsObj.find(text=lambda text:isinstance(text, Comment))
    if "saved from url" in origen: # puedo identificar desde donde se ha descargado la página
        ....
 

- Según el origen se identifican los *tag's* *html* que contienen tanto el titular de la noticia, para nombrar el *txt* resultante como el cuerpo de la noticia:

In [ ]:

    if "www.theguardian.com" in origen:
        hayQueTratar = True
        titulo = bsObj.find('h1', attrs={'class' : 'content__headline'}).text
        objBody = bsObj.find('div', attrs={'itemprop' : 'articleBody'})


en algún caso, además del titular, se extrae una segunda cabecera de la noticia, o se elimina información sobrante que la librería extrae junto con el texto:

In [ ]:

    elif "www.telegraph.co.uk" in origen:
        hayQueTratar = True
        titulo = bsObj.find('h1', attrs={'itemprop' : 'headline name'}).text
        objBody = bsObj.find('article', attrs={'itemprop': 'articleBody'})
        cad_inicio = '/* dynamic basic css */'
        cad_fin = 'OBR.extern.researchWidget();'
    elif "elpais.com" in origen:
        hayQueTratar = True
        titulo = bsObj.find('h1', attrs={'itemprop': 'headline'}).text
        subtitulo = bsObj.find('h2', attrs={'itemprop': 'alternativeHeadline'}).text
        objBody = bsObj.find('div', attrs={'itemprop': 'articleBody'})


- con la información recogida se guarda en una nueva carpeta las conversiones a *txt* de cada ficheros:

In [ ]:

    f2 = open(folderDestino + "/" + titulo + ".txt", "w")
    f2.write(titulo + "\n")

    if subtitulo != None:
        subtitulo = subtitulo.replace("\n", "")
        f2.write(subtitulo + "\n")

    if objBody != None:
        for parrafo in objBody.findAll('p'):
            aux = parrafo.text

            if cad_inicio != None and cad_fin != None:

                pos_inicio = aux.find(cad_inicio)
                pos_fin = aux.find(cad_fin) + len(cad_fin)

                if pos_inicio != -1 or pos_fin != -1:
                    aux = aux[:pos_inicio] + aux[pos_fin:]

            f2.write(aux + "\n")


#### - Lectura de los *txt* <a class="anchor" id="opcion-3-2"></a>

Se recoge cada uno de los *txt's* generados en el punto anterior y se cargan en memoria:

In [ ]:

    listing = os.listdir(folder + "/txt")
    for file in listing:

        if file.endswith(".txt"):
            url = folder+"/txt/"+file
            f = open(url,encoding="ANSI");
            raw = f.read()
            f.close()
            t = TextBlob(raw)
            idioma = t.detect_language()
            print("File: ", file," escrito en: ", idioma)
            
            raw_limpio = limpia_signos_puntuacion(raw)
            tokens = nltk.word_tokenize(raw_limpio)
            tokens_limpio = limpia_stop_words(tokens, idioma)


#### - Tratamiento de los textos <a class="anchor" id="opcion-3-3"></a>

Sobre los texto leídos se aplican los distintos métodos y herramientas descritos anteriormente:

In [ ]:

        #text = nltk.Text(stemming(tokens_limpio, idioma))

        #text = nltk.Text(lemmatization(tokens_limpio, idioma))

        #text = nltk.Text(stemming(lemmatization(tokens_limpio, idioma),idioma))

        #text = nltk.Text(trata_entity_names(tokens_limpio))

        #text = nltk.Text(stemming(trata_entity_names(tokens_limpio), idioma))


#### - Agrupación y resultado <a class="anchor" id="opcion-3-2"></a>

Tras haber aplicado cada uno de los métodos, se procesde a realizar la agrupación y la comparación con el array de soluciones optimo, obteniendo el porcentaje de exito del proceso:

In [ ]:

    distanceFunction ="cosine"
    #distanceFunction = "euclidean"
    test = cluster_texts(texts,5,distanceFunction)
    print("test: ", test)
    # Gold Standard
    reference =[0, 5, 0, 0, 0, 2, 2, 3, 5, 5, 5, 5, 5, 4, 4, 4, 4, 3, 2, 0, 2, 5]
    print("reference: ", reference)

    # Evaluation
    print("rand_score: ", adjusted_rand_score(reference,test))


### Conclusiones <a class="anchor" id="opcion-4"></a>

Dejando aparte el proceso de extracción del texto desde el HTML, los pasos dados para conseguir la mejor agrupación han sido:

- realizando una primera ejecución del proceso sin modificar el texto leído, obtenemos el siguiente resultado, siendo este el punto de partida y el que obtendrá la peor puntuación:

 - sobre el texto leído desde el *txt* se eliminan los signos de puntuación y las *stopwords*